In [11]:
import requests


In [111]:
import json

#Copying the headers from the network tab on chrome so that bovada thinks the request is real
def headers_to_json(headers_string):
    headers = {}
    lines = headers_string.strip().split("\n")
    for i in range(0, len(lines), 2):
        key = lines[i].strip()[:-1]
        value = lines[i+1].strip()
        headers[key] = value
    return headers

def extract_decimal(s: str) -> float:
    match = re.search(r"(\d+\.\d+)", s)
    if match:
        return float(match.group(1))
    return None

headers_string = '''
Accept:
application/json, text/plain, */*
Accept-Encoding:
gzip, deflate, br
Accept-Language:
en-US,en;q=0.9
Referer:
https://www.bovada.lv/
Sec-Ch-Ua:
"Not/A)Brand";v="99", "Google Chrome";v="115", "Chromium";v="115"
Sec-Ch-Ua-Mobile:
?0
Sec-Ch-Ua-Platform:
"macOS"
Sec-Fetch-Dest:
empty
Sec-Fetch-Mode:
cors
Sec-Fetch-Site:
same-origin
User-Agent:
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36
X-Channel:
desktop
X-Sport-Context:
FOOT
'''  
headers = headers_to_json(headers_string)


In [33]:
data = requests.get('https://www.bovada.lv/services/sports/event/coupon/events/A/description/football/nfl-season-player-props/season-player-specials?marketFilterId=rank&preMatchOnly=true&eventsLimit=5000&lang=en',headers=headers).json()

In [129]:
import pandas as pd


def extract_player_stat(line: str) -> tuple:
    
    # Regular expression to match lines that contain player names and stats
    pattern = r'^(?P<name>[A-Za-z. ]+) (?P<year>\d{4}-\d{2}) Regular Season Total (?P<stat>.+)$'

    match = re.match(pattern, line)
    if match:
        return match.group('name'), match.group('stat')
    return None


df = pd.DataFrame()

lst = []
for event in data[0]['events']:
    description = event['description']
    markets = event ['displayGroups'][0]['markets']
    for market in markets:
        market_description = market['description']
        outcomes = market['outcomes']
        
        if "Sacks" in market_description or "Tackles" in market_description:
            continue
        elif '2023-24 To' in market_description:
            d = {}
            d['stat'] = market_description.replace('2023-24 To','').strip()
            for outcome in outcomes:
                d['player_name'] = outcome['description']
                d['odds'] = outcome['price']['american']
                lst.append(d.copy())
        else:   
            d = {}
            play_stat_tup = extract_player_stat(market_description)
            if not play_stat_tup:
                continue
            name,stat = play_stat_tup
            d['player_name'] = name
            d['stat'] = stat
            for outcome in outcomes:
                outcome_description = outcome['description']
                if 'Over' in outcome_description:
                    d['line'] = extract_decimal(outcome_description)
                    d['odds'] = outcome['price']['american']
            lst.append(d)
        

In [130]:
pd.DataFrame(lst)

,stat,player_name,odds,line
0,Throw 30+Regular Season Passing TDs,Patrick Mahomes,-550,NaN
1,Throw 30+Regular Season Passing TDs,Josh Allen,-250,NaN
2,Throw 30+Regular Season Passing TDs,Joe Burrow,-250,NaN
3,Throw 30+Regular Season Passing TDs,Justin Herbert,-170,NaN
4,Throw 30+Regular Season Passing TDs,Kirk Cousins,-120,NaN
...,...,...,...,...
734,Rushing Yards,Najee Harris,-115,950.5
735,Rushing Yards,Nick Chubb,-115,1200.5
736,Rushing Yards,Rhamondre Stevenson,-115,850.5
737,Rushing Yards,Tony Pollard,-115,975.5


In [118]:
data[0]['events'][3]['displayGroups'][0]

{'id': '100-303',
 'description': 'Props',
 'defaultType': False,
 'alternateType': True,
 'markets': [{'id': '274032743',
   'descriptionKey': '2023-24 To Throw 30+Regular Season Passing TDs',
   'description': '2023-24 To Throw 30+Regular Season Passing TDs',
   'key': 'RANK-PROP',
   'marketTypeId': '153',
   'status': 'O',
   'singleOnly': True,
   'notes': '',
   'period': {'id': '119',
    'description': 'Game',
    'abbreviation': 'G',
    'live': False,
    'main': True},
   'outcomes': [{'id': '1396246097',
     'description': 'Patrick Mahomes',
     'status': 'O',
     'type': 'X',
     'price': {'id': '14125066816',
      'american': '-550',
      'decimal': '1.181818',
      'fractional': '2/11',
      'malay': '0.18',
      'indonesian': '-5.50',
      'hongkong': '0.18'}},
    {'id': '1396246098',
     'description': 'Josh Allen',
     'status': 'O',
     'type': 'X',
     'price': {'id': '14125066818',
      'american': '-250',
      'decimal': '1.400',
      'fractional

In [109]:
for event in data[0]['events']:
    for market in event['displayGroups'][0]['markets'][:5]:
        print(market['description'])
        if 
    print('')

Aaron Jones Rushing & Receiving Yards
Austin Ekeler Rushing & Receiving Yards
Christian McCaffrey Rushing & Receiving Yards
Jahmyr Gibbs Rushing & Receiving Yards
James Cook Rushing & Receiving Yards

Alex Singleton 2023-24 Regular Season Total Combined Tackles
Bobby Okereke 2023-24 Regular Season Total Combined Tackles
Bobby Wagner 2023-24 Regular Season Total Combined Tackles
C.J. Mosley 2023-24 Regular Season Total Combined Tackles
Demario Davis 2023-24 Regular Season Total Combined Tackles

Asante Samuel Jr 2023 -24 Regular Season Defensive Interceptions
Charvarius Ward 2023 -24 Regular Season Defensive Interceptions
Christian Gonzalez 2023 -24 Regular Season Defensive Interceptions
Darius Slay 2023 -24 Regular Season Defensive Interceptions
Denzel Ward 2023 -24 Regular Season Defensive Interceptions

2023-24 To Throw 30+Regular Season Passing TDs
2023-24 To Throw 35+Regular Season Passing TDs
Aaron Rodgers 2023-24 Regular Season Total Passing TDs
Anthony Richardson 2023-24 Regular

{'id': '100-303',
 'description': 'Props',
 'defaultType': False,
 'alternateType': True,
 'markets': [{'id': '274034605',
   'descriptionKey': 'Aaron Jones Rushing & Receiving Yards',
   'description': 'Aaron Jones Rushing & Receiving Yards',
   'key': 'RANK-PROP',
   'marketTypeId': '153',
   'status': 'O',
   'singleOnly': True,
   'notes': '',
   'period': {'id': '119',
    'description': 'Game',
    'abbreviation': 'G',
    'live': False,
    'main': True},
   'outcomes': [{'id': '1396251118',
     'description': 'Over 1350.5',
     'status': 'O',
     'type': 'X',
     'price': {'id': '14126784224',
      'american': '+115',
      'decimal': '2.150',
      'fractional': '23/20',
      'malay': '-0.87',
      'indonesian': '1.15',
      'hongkong': '1.15'}},
    {'id': '1396251119',
     'description': 'Under 1350.5',
     'status': 'O',
     'type': 'X',
     'price': {'id': '14126784223',
      'american': '-150',
      'decimal': '1.666667',
      'fractional': '2/3',
      'ma

In [110]:
import re

def extract_player_stat(line: str) -> tuple:
    # Regular expression to match lines that contain player names and stats
    pattern = r'^(?P<name>[A-Za-z. ]+) (?P<year>\d{4}-\d{2}) Regular Season Total (?P<stat>.+)$'

    match = re.match(pattern, line)
    if match:
        return match.group('name'), match.group('stat')
    return None

# Testing the function with your example lines:
lines = [
    "Aaron Jones Rushing & Receiving Yards",
    "Alex Singleton 2023-24 Regular Season Total Combined Tackles",
    "Bobby Wagner 2023-24 Regular Season Total Combined Tackles",
    "Asante Samuel Jr 2023 -24 Regular Season Defensive Interceptions",
    "Aaron Rodgers 2023-24 Regular Season Total Passing TDs",
    "2023-24 To Have 4000+ Regular Season Passing Yards",
    "A.J. Brown 2023-24 Regular Season Total Receiving TDs",
    "A.J. Brown 2023-24 Regular Season Total Receiving Yards",
    "Adam Thielen 2023-24 Regular Season Total Receptions",
    "A.J. Dillon 2023-24 Regular Season Total Rushing TDs",
    "Nick Bosa 2023-24 Regular Season Total Sacks",
    "Any Player to Break the Record for Most Def Sacks"
]

for line in lines:
    result = extract_player_stat(line)
    if result:
        print(result)

('Alex Singleton', 'Combined Tackles')
('Bobby Wagner', 'Combined Tackles')
('Aaron Rodgers', 'Passing TDs')
('A.J. Brown', 'Receiving TDs')
('A.J. Brown', 'Receiving Yards')
('Adam Thielen', 'Receptions')
('A.J. Dillon', 'Rushing TDs')
('Nick Bosa', 'Sacks')


In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time

# Set up Selenium WebDriver
driver = webdriver.Safari()

# Open the URL
url = "https://www.bovada.lv/sports/football/nfl-season-player-props/season-player-specials/futures-odd"
driver.get(url)

time.sleep(5)

# Get the HTML content
html_content = driver.page_source

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html_content, 'html.parser')

time.sleep(5)

# Create a DataFrame to store the data
data = {
    'Player Name': [],
    'Over Line': [],
    'Over Price': [],
    'Under Line': [],
    'Under Price': []
}

# Iterate through each market section
for market in soup.find_all('div', class_='score-container'):
    player_name = market.find('h3', class_='market-name').text.strip()
    over_line = market.find_all('li', class_='other')[0].find('span', class_='outcomes').text.split()[-1]
    over_price = market.find_all('li', class_='other')[0].find('span', class_='bet-price').text.strip()
    under_line = market.find_all('li', class_='other')[1].find('span', class_='outcomes').text.split()[-1]
    under_price = market.find_all('li', class_='other')[1].find('span', class_='bet-price').text.strip()

    # Append to data
    data['Player Name'].append(player_name)
    data['Over Line'].append(over_line)
    data['Over Price'].append(over_price)
    data['Under Line'].append(under_line)
    data['Under Price'].append(under_price) 

# Create a DataFrame
df = pd.DataFrame(data)

# Function to split player name and stat
def split_player_stat(row):
    # Define stat categories
    stat_categories = [
        "Rushing & Receiving Yards", "Receiving Yards", "Passing TDs",
        "Receiving TDs", "Rushing Yards", "Rushing TDs", "Total Interceptions", 
        "Passing Yards"
    ]
    for stat in stat_categories:
        if stat in row['Player Name']:
            player = row['Player Name'].replace(stat, '').replace('2023-24 Regular Season', '').replace('Total', '').strip()
            return pd.Series([player, stat])
    return pd.Series([row['Player Name'], None])

# Split "Player Name" into "Player" and "Stat" columns
df[['Player', 'Stat']] = df.apply(split_player_stat, axis=1)

# Filter out rows with None in the "Stat" column (if any)
df = df.dropna(subset=['Stat'])

# Replace "EVEN" with 100
df.replace('EVEN', '100', inplace=True)

# Reorder and select the desired columns
df = df[['Player', 'Stat', 'Over Line', 'Over Price', 'Under Line', 'Under Price']]

# Save to CSV
df.to_csv('players.csv', index=False)

# Close the WebDriver
driver.quit()
